# Hava durumu modeli

In [6]:
import numpy as np
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
from sklearn import linear_model
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, r2_score
import statsmodels.api as sm

import warnings
warnings.filterwarnings('ignore')

baslik_font = {'family': 'arial','color':  'darkred','weight': 'bold','size': 15 }
eksen_font = {'family': 'arial','color':  'darkblue','weight': 'bold','size': 10 }

In [1]:

weather_df = pd.read_csv("https://djl-lms-assets.s3.eu-central-1.amazonaws.com/datasets/weatherHistory.csv")
weather_df.head()

,Formatted Date,Summary,Precip Type,Temperature (C),Apparent Temperature (C),Humidity,Wind Speed (km/h),Wind Bearing (degrees),Visibility (km),Loud Cover,Pressure (millibars),Daily Summary
0,2006-04-01 00:00:00.000 +0200,Partly Cloudy,rain,9.472222,7.388889,0.89,14.1197,251.0,15.8263,0.0,1015.13,Partly cloudy throughout the day.
1,2006-04-01 01:00:00.000 +0200,Partly Cloudy,rain,9.355556,7.227778,0.86,14.2646,259.0,15.8263,0.0,1015.63,Partly cloudy throughout the day.
2,2006-04-01 02:00:00.000 +0200,Mostly Cloudy,rain,9.377778,9.377778,0.89,3.9284,204.0,14.9569,0.0,1015.94,Partly cloudy throughout the day.
3,2006-04-01 03:00:00.000 +0200,Partly Cloudy,rain,8.288889,5.944444,0.83,14.1036,269.0,15.8263,0.0,1016.41,Partly cloudy throughout the day.
4,2006-04-01 04:00:00.000 +0200,Mostly Cloudy,rain,8.755556,6.977778,0.83,11.0446,259.0,15.8263,0.0,1016.51,Partly cloudy throughout the day.



1. Hedef değişkeninizin görünür sıcaklık ve sıcaklık arasındaki fark olduğu doğrusal bir regresyon modeli oluşturun. 
2. Açıklayıcı değişkenler olarak nem ve rüzgar hızı kullanın. Şimdi, modelinizi OLS kullanarak tahmin edin.

In [3]:
weather_df.columns

Index(['Formatted Date', 'Summary', 'Precip Type', 'Temperature (C)',
       'Apparent Temperature (C)', 'Humidity', 'Wind Speed (km/h)',
       'Wind Bearing (degrees)', 'Visibility (km)', 'Loud Cover',
       'Pressure (millibars)', 'Daily Summary'],
      dtype='object')

In [7]:
Y = weather_df["Temperature (C)"] - weather_df["Apparent Temperature (C)"]
X = weather_df[["Humidity","Wind Speed (km/h)"]]

In [8]:
lrm = linear_model.LinearRegression()
lrm.fit(X, Y)

LinearRegression()

In [9]:
print('Değişkenler: \n', lrm.coef_)
print('Sabit değer (bias): \n', lrm.intercept_)

Değişkenler: 
 [3.02918594 0.11929075]
Sabit değer (bias): 
 -2.4381054151877013


In [10]:
X = sm.add_constant(X)
results = sm.OLS(Y, X).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.288
Model:                            OLS   Adj. R-squared:                  0.288
Method:                 Least Squares   F-statistic:                 1.949e+04
Date:                Tue, 11 May 2021   Prob (F-statistic):               0.00
Time:                        11:27:16   Log-Likelihood:            -1.7046e+05
No. Observations:               96453   AIC:                         3.409e+05
Df Residuals:                   96450   BIC:                         3.409e+05
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                -2.4381      0.021   -115.948      0.000      -2.479      -2.397
Humidity              3.0292      0.024    126.479      0.000       2.982       3.076
Wind Speed (km/h)     0.1193      0.001    176.164      0.000       0.118       0.121
==============================================================================
Omnibus:                     3935.747   Durbin-Watson:                   0.264
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             4613.311
Skew:                           0.478   Prob(JB):                         0.00
Kurtosis:                       3.484   Cond. No.                         88.1
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

# Tahmin edilen katsayılar istatistiksel olarak anlamlı mıdır?

P-Değeri 0 çıktığı için istatistiksel olarak anlamlıldır.

# Tahmini katsayılar önceki beklentileriniz doğrultusunda mı? Tahmin edilen katsayıları yorumlayınız. Hedef ve açıklayıcı değişkenler arasındaki ilişkiler nelerdir?

Rüzgar hızının etkisinin az olacağını düşünüyordum, katsayısı çok düşük geldi, beklediğim gibi. Ek olarak Nem parametresinin de katsayısı oldukça düşük. 1 birimlik rüzgar artış hızı modele çok fazla bir etki sunmuyor.

In [15]:
Y = weather_df["Temperature (C)"] - weather_df["Apparent Temperature (C)"]
weather_df["Humidity_WindSpeed"] = weather_df["Humidity"] * weather_df["Wind Speed (km/h)"]
X = weather_df[["Humidity","Wind Speed (km/h)","Humidity_WindSpeed"]]

X = sm.add_constant(X)

sonuclar = sm.OLS(Y, X).fit()
sonuclar.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.341
Model:                            OLS   Adj. R-squared:                  0.341
Method:                 Least Squares   F-statistic:                 1.666e+04
Date:                Tue, 11 May 2021   Prob (F-statistic):               0.00
Time:                        12:02:03   Log-Likelihood:            -1.6669e+05
No. Observations:               96453   AIC:                         3.334e+05
Df Residuals:                   96449   BIC:                         3.334e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
======================================================================================
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                 -0.0839      0.033     -2.511      0.012      -0.149      -0.018
Humidity              -0.1775      0.043     -4.133      0.000      -0.262      -0.093
Wind Speed (km/h)     -0.0905      0.002    -36.797      0.000      -0.095      -0.086
Humidity_WindSpeed     0.2971      0.003     88.470      0.000       0.291       0.304
==============================================================================
Omnibus:                     4849.937   Durbin-Watson:                   0.262
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             9295.404
Skew:                           0.378   Prob(JB):                         0.00
Kurtosis:                       4.320   Cond. No.                         193.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Katsayılar istatistiksel olarak anlamlıdır. Nem ve hız birlikte eksi değer almıştır. Öncesinde birlikte pozitif değerlere sahiptiler. Birbirleri ile doğrusal bir ilişkisi olduğu gözükmektedir.

# Ev fiyatları modeli

In [16]:
# df = house_prices_df
df = pd.read_csv("https://djl-lms-assets.s3.eu-central-1.amazonaws.com/datasets/house_prices.csv", sep = ";")
df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [23]:
Y3 = df["SalePrice"]
X3 = df[["OverallQual","GrLivArea","GarageCars","GarageArea","TotalBsmtSF","1stFlrSF","FullBath","YearBuilt","YearRemodAdd"]]

In [24]:
lrm3 = linear_model.LinearRegression()
lrm3.fit(X3, Y3)

LinearRegression()

In [25]:
X3 = sm.add_constant(X3)
results = sm.OLS(Y3, X3).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              SalePrice   R-squared:                       0.774
Model:                            OLS   Adj. R-squared:                  0.772
Method:                 Least Squares   F-statistic:                     550.8
Date:                Tue, 11 May 2021   Prob (F-statistic):               0.00
Time:                        15:44:33   Log-Likelihood:                -17459.
No. Observations:                1460   AIC:                         3.494e+04
Df Residuals:                    1450   BIC:                         3.499e+04
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const        -1.186e+06   1.28e+05     -9.241      0.000   -1.44e+06   -9.34e+05
OverallQual    1.96e+04   1187.658     16.505      0.000    1.73e+04    2.19e+04
GrLivArea       51.3821      3.107     16.536      0.000      45.287      57.477
GarageCars    1.043e+04   3032.638      3.438      0.001    4476.579    1.64e+04
GarageArea      14.9263     10.283      1.452      0.147      -5.245      35.097
TotalBsmtSF     19.8561      4.283      4.636      0.000      11.454      28.258
1stFlrSF        14.1741      4.928      2.876      0.004       4.507      23.841
FullBath     -6780.2962   2657.525     -2.551      0.011    -1.2e+04   -1567.291
YearBuilt      268.1205     50.165      5.345      0.000     169.717     366.524
YearRemodAdd   296.4379     63.596      4.661      0.000     171.688     421.188
==============================================================================
Omnibus:                      478.177   Durbin-Watson:                   1.985
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            58969.223
Skew:                          -0.413   Prob(JB):                         0.00
Kurtosis:                      34.124   Cond. No.                     4.67e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 4.67e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

## Ev fiyatları modelinizi tekrar çalıştırın ve sonuçları yorumlayın. Hangi özellikler istatistiksel olarak anlamlı ve hangileri değildir?

GarageArea özelliği o.1den büyük olduğundan istatistiksel olarak anlamlı değildir. Onun dışındaki diğer tüm özellikler anlamlaıdır.

# Şimdi, anlamsız özellikleri modelinizden hariç tutun. Bir şey değişti mi?

In [27]:
Y3 = df["SalePrice"]
X3 = df[["OverallQual","GrLivArea","GarageCars","TotalBsmtSF","1stFlrSF","FullBath","YearBuilt","YearRemodAdd"]]

In [28]:
lrm3 = linear_model.LinearRegression()
lrm3.fit(X3, Y3)

LinearRegression()

In [29]:
X3 = sm.add_constant(X3)
results = sm.OLS(Y3, X3).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              SalePrice   R-squared:                       0.773
Model:                            OLS   Adj. R-squared:                  0.772
Method:                 Least Squares   F-statistic:                     618.9
Date:                Tue, 11 May 2021   Prob (F-statistic):               0.00
Time:                        15:47:08   Log-Likelihood:                -17460.
No. Observations:                1460   AIC:                         3.494e+04
Df Residuals:                    1451   BIC:                         3.499e+04
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const        -1.188e+06   1.28e+05     -9.255      0.000   -1.44e+06   -9.36e+05
OverallQual   1.959e+04   1188.062     16.486      0.000    1.73e+04    2.19e+04
GrLivArea       51.7669      3.097     16.714      0.000      45.692      57.842
GarageCars    1.395e+04   1816.623      7.680      0.000    1.04e+04    1.75e+04
TotalBsmtSF     20.3847      4.269      4.775      0.000      12.010      28.759
1stFlrSF        14.6512      4.919      2.979      0.003       5.002      24.300
FullBath     -7183.7785   2643.957     -2.717      0.007   -1.24e+04   -1997.392
YearBuilt      269.8900     50.169      5.380      0.000     171.478     368.302
YearRemodAdd   295.7406     63.618      4.649      0.000     170.947     420.534
==============================================================================
Omnibus:                      462.446   Durbin-Watson:                   1.984
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            54365.371
Skew:                          -0.355   Prob(JB):                         0.00
Kurtosis:                      32.886   Cond. No.                     4.62e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 4.62e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

Diğer özelliklerin hepsinin katsayısı arttı.

# Ev fiyatları ile olan ilişkilerini belirleyerek istatistiksel olarak anlamlı katsayıları yorumlayın. Ev fiyatları üzerinde hangi özellikler daha belirgin bir etkiye sahiptir?

En belirgin etkiye sahip olan özellikler,
1. OverallQual
2. GarageCars
3. GrLivArea

# Sonuçlar size mantıklı geliyor mu? Değilse, potansiyel nedenleri açıklamaya çalışın.

Sonuçlar mantıklı.